In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym.envs.registration import register
import time
import os
from .main import egas, computeQValue, decay, punish


In [3]:
env = gym.make('CartPole-v1', render_mode="human")
env.reset()

(array([-0.04619767, -0.00790047, -0.03352229,  0.02361843], dtype=float32),
 {})

In [4]:
env.reset()
for _ in range(100):
    env.render()
    observation, reward, done, info, check = env.step(env.action_space.sample())
    time.sleep(0.03)
    if done:
        env.reset()
env.close()

C:\Users\bherw\OneDrive\Desktop\Suraj\Reinforment Learning\gridworld\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
EPOCHS = 20000
ALPHA = 0.8
GAMMA = 0.9

epsilon = 1.0
min_epsilon = 0.01
max_epsilon = 1.0
decay_val = 0.0045

In [ ]:
def createBins(noOfBins):
    binCartPosition = np.linspace(-4.8, 4.8, noOfBins)
    binCartVelocity = np.linspace(-5, 5, noOfBins)
    binPoleAngle = np.linspace(-0.418, 0.418, noOfBins)
    binAngularVelocity = np.linspace(-5, 5, noOfBins)

    return np.array([binCartPosition, binCartVelocity, binPoleAngle, binAngularVelocity])

In [ ]:
def discretizeObs(observation, bins):


In [ ]:
for epoch in range(EPOCHS):
    initState = env.reset()